In [102]:
import pandas as pd
import random
import tensorflow as tf 

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split

In [103]:
siameseData = pd.read_csv("data.txt", sep= ';', index_col=None, names= ["text"], header=None)
siameseData

,text
0,Similarity has always been a key aspect in com...
1,"Any time two element vectors are compared, man..."
2,But if the comparison has to be applied to mor...
3,"In these cases, a siamese neural network may b..."
4,The two neural networks are both feedforward p...
5,they work parallelly in tandem and compare th...
6,The output generated by a siamese neural netwo...
7,In this overview we first describe the siamese...
8,"Additionally, we list the programming language..."
9,This section reviews existing tracking method...


In [90]:
siameseData["target"] = 1

In [91]:
siameseData.head(10)

,text,target
0,Similarity has always been a key aspect in com...,1
1,"Any time two element vectors are compared, man...",1
2,But if the comparison has to be applied to mor...,1
3,"In these cases, a siamese neural network may b...",1
4,The two neural networks are both feedforward p...,1
5,they work parallelly in tandem and compare th...,1
6,The output generated by a siamese neural netwo...,1
7,In this overview we first describe the siamese...,1
8,"Additionally, we list the programming language...",1
9,This section reviews existing tracking method...,1


In [92]:
randomData = pd.read_csv("random.txt", sep= ';', index_col=None, names= ["text"], header=None)

In [93]:
randomData["target"] = 0

In [94]:
train_df = pd.concat([randomData, siameseData], ignore_index=True )
train_df

,text,target
0,The Views section describes all you need to kn...,0
1,"Views consist of an app template, a layout, a...",0
2,ou can define a default layout by adding a de...,0
3,vue file inside the layouts directory,0
4,This will be used for all pages that don't ha...,0
...,...,...
78,One is recognizing a person among a large numb...,1
79,DeepFace is an example of such a system,1
80,In its most extreme form this is recognizing a...,1
81,"The other is face verification, that is to ver...",1


In [96]:
train_shuffle=train_df.sample(frac=1,random_state=49) 
train_shuffle

,text,target
25,"Derived from the Wichianmat landrace, one of ...",0
74,This is typically used for comparing similar i...,1
21,” The “unrolled” visual represents the individ...,0
82,"The twin network might be the same, but the im...",1
57,"Recently, the DSiamM tracker proposes to perfo...",1
...,...,...
12,they are incorporated into popular applicatio...,0
54,The matching function is usually implemented b...,1
40,The output generated by a siamese neural netwo...,1
45,"Recently, Danelljan et al",1


In [120]:
train_sentences, test_sentences, train_labels,test_labels = train_test_split(
    train_shuffle['text'].to_numpy(),
    train_shuffle['target'].to_numpy(),
    test_size=0.1, 
    random_state=49
)

In [122]:
text_vectorizer = TextVectorization(max_tokens=10000, 
                                    standardize="lower_and_strip_punctuation", 
                                    output_sequence_length=15)

In [106]:
text_vectorizer.adapt(train_sentences)

2021-11-04 18:09:29.430714: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [111]:
words = text_vectorizer.get_vocabulary()

embedding = Embedding(
                        input_dim=10000, 
                        output_dim=128, 
                        input_length=15, 
                        name = 'embeding_1'
)

In [113]:
words[:10]

['', '[UNK]', 'the', 'a', 'to', 'of', 'is', 'and', 'in', 'network']

In [117]:
words[-10:]

['adopts',
 'adopt',
 'adaptation',
 'actual',
 'achieves',
 'about',
 'able',
 'ability',
 '2',
 '1994']

In [118]:
test_string = "twin neural network"

In [119]:
sample_embed = embedding(text_vectorizer(test_string))
sample_embed

<tf.Tensor: shape=(15, 128), dtype=float32, numpy=
array([[-0.03797289, -0.01955438, -0.00317963, ..., -0.01730888,
         0.03729964, -0.00263832],
       [-0.03407349, -0.03599926, -0.04305427, ..., -0.02267176,
        -0.0042868 , -0.03615265],
       [ 0.03482663,  0.0257888 ,  0.04839382, ...,  0.03812052,
        -0.02405619,  0.04215841],
       ...,
       [-0.03748167,  0.00327482, -0.04108505, ..., -0.01556055,
         0.04547551,  0.0313192 ],
       [-0.03748167,  0.00327482, -0.04108505, ..., -0.01556055,
         0.04547551,  0.0313192 ],
       [-0.03748167,  0.00327482, -0.04108505, ..., -0.01556055,
         0.04547551,  0.0313192 ]], dtype=float32)>